Modified: Jul 31, 2019
# hv.Polydraw linked stream

In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys
import numpy as np
import scipy as sp
import pandas as pd
import intake
    
from pathlib import Path
from pprint import pprint
from sklearn.externals import joblib
import pdb

import matplotlib.pyplot as plt
%matplotlib inline

# ignore warnings
import warnings
if not sys.warnoptions:
    warnings.simplefilter('ignore')
    
# Don't generate bytecode
sys.dont_write_bytecode = True

import holoviews as hv
import panel as pn
from holoviews import opts
hv.notebook_extension('bokeh')

In [ ]:
from holoviews.streams import *

In [ ]:
H,W = 450,450
opts.defaults(
    opts.Image(width=W, height=H, padding=0.1,
               active_tools=['poly_draw','wheel_zoom']),
    opts.Path(color='green', line_width=3, 
            width=W, height=H, padding=0.1),
    opts.Polygons(fill_alpha=0.3, active_tools=['poly_draw', 'wheel_zoom'], 
                 width=W, height=H, padding=0.1)
)

## Set up a debug box using `param.watch`
- refer to this excellent [tutorial](https://holoviz.org/tutorial/Interlinked_Panels.html) on how to add callbacks to param.parameter objects

In [ ]:
mdbox = pn.pane.Markdown('')
# mdbox.object = "Hi!"

In [ ]:
def echo_event(event):
    output = f"""#Event log
    {event}
    """
    mdbox.object = output

In [ ]:
slider = pn.widgets.IntSlider(value=0, start=0, end=10)


In [ ]:
slider.param.watch(echo_event, 'value')

In [ ]:
pn.Row(slider, mdbox)

## Create a hv.Polygon element and add PolyDraw stream 
which receives the data from these elements

In [ ]:
poly = hv.Polygons([(0,0),(2,0),(1,2)])
empty_plot = hv.Polygons([])
stream_from_poly = PolyDraw(source=poly, num_objects=10, drag=True, show_vertices=True)
stream_from_plot = PolyDraw(source=empty_plot, num_objects=10, drag=True, show_vertices=True)
poly + empty_plot

In [ ]:
# poly_stream.param.watch(echo_event, 'data')
stream_from_plot.param.watch(echo_event, 'data')

In [ ]:
pn.Row(empty_plot,  mdbox)

In [ ]:
# We can access the coordinates of the drawn polygon from the stream
stream_from_plot.data

## PolyDraw stream on hv.Image
Let's see how it works with the underlying cartesian coordinate system of an hv.Image object. This will be useful for the levelset propagation interaction gui for segmentation.

In [ ]:
from skimage import io, img_as_float, data

In [ ]:
img = img_as_float(data.binary_blobs())
img.min(), img.max()

In [ ]:
hvimg = hv.Image(img, bounds=(0,0,512,512))
polyplot = hv.Polygons([])

In [ ]:
draw_stream = PolyDraw(source=polyplot, num_objects=10, drag=True, show_vertices=True)
draw_stream.param.watch(echo_event, 'data')
# pn.Row(hvimg*polyplot, mdbox)

In [ ]:
# Access the polydraw's polygon coordinates from the PolyDraw stream object
draw_stream.data

In [ ]:
@pn.depends(draw_stream.param.data)
def show_polydraw(data):
    return hv.Table(data, ['xs','ys'])

In [ ]:
pn.Row(hvimg*polyplot, 
       pn.Column(show_polydraw, mdbox)
      )